In this notebook, we will demonstrate how arithmetic operation between paramaters work and what is propagated, as well as the use-case of making a dependent parameter. But first, a description of `DescriptorNumbers` and `Parameters`

In [2]:
import numpy as np
from easyscience import Parameter
from easyscience import DescriptorNumber

# `DescriptorNumbers` and `Parameters`

`DescriptorNumbers` and `Parameters` are the main components of models in `EasyScience`. They contain a value with a variance, unit and other fields.
Lets look at the constructor for a `DescriptorNumber`:

In [43]:
a = DescriptorNumber(
    name='a',
    value=1.0,
    unit='m',
    variance=0.1,
    description='Length of the side of a square',
    display_name='Side Length',
    url='https://example.com/side_length',
    unique_name='length_of_square',
)

Internally, we use Scipp scalars to hold the value, unit and variance. So a `DescriptorNumber` is just a Scipp scalar with a few extra fields mostly relevant for GUI applications:

The name is the name of `DescriptorNumber`, this is a mandatory attribute which will be removed in the future

The value is the only other mandatory attribute.

Description and url are relevant to GUI tooltips but otherwise have little importance.

Display_name is a prettier formatted name used for displaying the `DescriptorNumber` both in the GUI and in other representations.

The unique_name is a unique identifier for the `DescriptorNumber`, which by default is autogenerated as the class name underscore a number: ´DescriptorNumber_0`, but it can also be chosen manually. Although it HAS to be unique. Will raise an error if not unique.

Now lets look at the `Parameter` construsctor

In [44]:
b = Parameter(
    name='b',
    value=2.0,
    unit='m',
    variance=0.2,
    min=0.0,
    max=10.0,
    fixed=False,
    description='Length of the diagonal of a square',
    display_name='Diagonal Length',
    url='https://example.com/diagonal_length',
    unique_name='diagonal_of_square',
)

We can see that it is just like the `DescriptorNumber` except for 2 more fields:

Min/max, which defines the lower and upper bounds that the Parameter value can assume.

Fixed, which is a flag telling whether the Parameter should be fixed to a specific value during fitting.

# Arithmetic Operations

## Addition/Subtraction

First, we make some Parameters and DescriptorNumbers.

In [15]:
a = Parameter(name="a", value=1.0, variance=0.01, unit="m", min=0, max=10)
b = Parameter(name="b", value=2.0, variance=0.1, unit="m", min=-10, max=10)
c = DescriptorNumber(name="c", value=3.0, variance=0.1, unit="m")
d = DescriptorNumber(name="d", value=4.0, variance=1, unit="m")

Lets start simple with addition and subtraction, note how the bounds are also properly propagated

In [50]:
a+b

<Parameter 'Parameter_56': 3.0000 ± 0.3317 m, bounds=[-10.0:20.0]>

Addition/Subtraction of `Parameters` and `DescriptorNumbers` yield a `Parameter`, because the result is varyable.

In [51]:
a-c

<Parameter 'Parameter_57': -2.0000 ± 0.3317 m, bounds=[-3.0:7.0]>

Addition/Subtraction of two `DescriptorNumbers` yield a new `DescriptorNumber`, since the result is also not varyable.

In [52]:
c+d

<DescriptorNumber 'DescriptorNumber_6': 7.0000 ± 1.0488 m>

We use Scipp under the hood, but with some small changes, one of these changes is the addition/subtraction of Parameters with the same dimension but different prefixes:

In [16]:
a = Parameter(name="a", value=1.0, variance=0.01, unit="m", min=0, max=10)
b = Parameter(name="b", value=2.0, variance=0.1, unit="cm", min=-10, max=10)
from scipp import scalar
a_scipp = scalar(a.value, variance=a.variance, unit=a.unit)
b_scipp = scalar(b.value, variance=b.variance, unit=b.unit)

In [21]:
a+b

<Parameter 'Parameter_18': 1.0200 ± 0.1000 m, bounds=[-0.1:10.1]>

In [55]:
a_scipp+b_scipp

UnitError: Cannot add m and cm.

Since we use Scipp internally, for error propagation, we convert the 2nd parameter to the unit of the first before doing the addition/subtraction.

## Multiplication

Next up is multiplication, again the bounds are properly propagated:

In [56]:
a = Parameter(name="a", value=1.0, variance=0.01, unit="m", min=-np.inf, max=np.inf)
b = Parameter(name="b", value=2.0, variance=0.1, unit="m", min=-10, max=10)
c = Parameter(name="c", value=3.0, variance=0.1, unit="s", min=0, max=np.inf)
d = Parameter(name="d", value=-4, variance=1, unit="kg", min=-50, max=0)
e = DescriptorNumber(name="e", value=5, variance=1, unit="Joule")

We can multiply by pure numbers.

In [57]:
2*b

<Parameter 'Parameter_65': 4.0000 ± 0.6325 m, bounds=[-20.0:20.0]>

And we can multiply by other Parameters and DescriptorNumbers.
Note how bounds are properly propagated and units are exponentiated:

In [58]:
a*b

<Parameter 'Parameter_66': 2.0000 ± 0.3742 m^2, bounds=[-inf:inf]>

Different units can also be multiplied together:

In [59]:
a*c

<Parameter 'Parameter_67': 3.0000 ± 0.4359 m*s, bounds=[-inf:inf]>

Bounds are properly propagated, even when one is only positive or negative

In [60]:
b*c

<Parameter 'Parameter_68': 6.0000 ± 1.1402 m*s, bounds=[-inf:inf]>

And just to show that the new bounds are not always `-np.inf` to `np.inf`

In [61]:
b*d

<Parameter 'Parameter_69': -8.0000 ± 2.3664 m*kg, bounds=[-500.0:500.0]>

And multiplication between `Parameters` and `DescriptorNumbers` yield `Parameters`

In [62]:
a*e

<Parameter 'Parameter_70': 5.0000 ± 1.1180 J*m, bounds=[-inf:inf]>

Multiplication of different pre-fixes yields a new pre-fix which is the middle-ground betweehn those pre-fixes:

In [63]:
b = Parameter(name="b", value=2.0, variance=0.1, unit="cm", min=-10, max=10)

In [64]:
a*b

<Parameter 'Parameter_72': 2.0000 ± 0.3742 dm^2, bounds=[-inf:inf]>

If there is no middle-ground prefix, the resulting unit is the unit closest to the base unit of the operands, this differs from how it is done in Scipp, where the units gets a decimal:

In [65]:
b = Parameter(name="b", value=2.0, variance=0.1, unit="dm", min=-10, max=10)
a_scipp = scalar(a.value, variance=a.variance, unit=a.unit)
b_scipp = scalar(b.value, variance=b.variance, unit=b.unit)

In [66]:
a*b

<Parameter 'Parameter_74': 0.2000 ± 0.0374 m^2, bounds=[-inf:inf]>

In [67]:
a_scipp*b_scipp

<scipp.Variable> ()    float64         [0.1m^2]  2  0.14

## Division

We can also divide our parameters:

In [68]:
a/b

<Parameter 'Parameter_75': 5.0000 ± 0.9354, bounds=[-inf:inf]>

In [69]:
b/d

<Parameter 'Parameter_76': -0.5000 ± 0.1479 dm/kg, bounds=[-inf:inf]>

And again, bounds often become `-np.inf` to `np.inf`, but not necessarily:

In [70]:
a = Parameter(name="a", value=1.0, variance=0.01, unit="m", min=1, max=10)
b = Parameter(name="b", value=2.0, variance=0.1, unit="s", min=2, max=10)

In [71]:
a/b

<Parameter 'Parameter_79': 0.5000 ± 0.0935 m/s, bounds=[0.1:5.0]>

Another import difference from Scipp here comes from division with 0, either as a number or as a `Parameter` value. In EasyScience, this results in a ZeroDivisionError:

In [72]:
b = Parameter(name="b", value=0.0, variance=0.1, unit="cm", min=-10, max=10)

In [73]:
a/b

ZeroDivisionError: Cannot divide by zero

In Scipp, this results in `np.inf`

In [76]:
a_scipp = scalar(a.value, variance=a.variance, unit=a.unit)
b_scipp = scalar(b.value, variance=b.variance, unit=b.unit)

In [77]:
a_scipp/b_scipp

<scipp.Variable> ()    float64            [1/%]  inf  inf

This is because in EasyScience we can't fit `NaN`s and infinities, so we avoid those values at all costs.

Compounded units are also automatically recognized:

In [78]:
a = Parameter(name="a", value=1.0, variance=0.01, unit="m", min=1, max=10)
b = Parameter(name="b", value=2.0, variance=0.1, unit="s", min=2, max=10)
c = Parameter(name="c", value=3.0, variance=0.1, unit="kg", min=0, max=np.inf)

In [79]:
c*a**2/b**2

<Parameter 'Parameter_87': 0.7500 ± 0.2915 J, bounds=[0.0:inf]>

# Dependent Parameters

In `EasyScience` we can now make dependent parameters, i.e. parameters whose values, units etc. are calculated from other `Parameters`, using these arithmetic operations

In [3]:
a = Parameter(name="a", value=1.0, variance=0.01, unit="m", min=1, max=10)
b = Parameter.from_dependency(name="b", dependency_expression="a**2", dependency_map={"a": a})
a,b

(<Parameter 'a': 1.0000 ± 0.1000 m, bounds=[1.0:10.0]>,
 <Parameter 'b': 1.0000 ± 0.2000 m^2, bounds=[1.0:100.0]>)

The dependent parameters values are automatically changed when their dependencies are updated:

In [4]:
a.value = 2.0
a,b

(<Parameter 'a': 2.0000 ± 0.1000 m, bounds=[1.0:10.0]>,
 <Parameter 'b': 4.0000 ± 0.4000 m^2, bounds=[1.0:100.0]>)

Unit conversions are also propagated:

In [5]:
a.convert_unit("cm")
a,b

(<Parameter 'a': 200.0000 ± 10.0000 cm, bounds=[100.0:1000.0]>,
 <Parameter 'b': 40000.0000 ± 4000.0000 cm^2, bounds=[10000.0:1000000.0]>)

You can also do logic dependencies:

In [6]:
b = Parameter.from_dependency(name="b", dependency_expression="a if a.value < 0 else a**2", dependency_map={"a": a})
a,b

(<Parameter 'a': 200.0000 ± 10.0000 cm, bounds=[100.0:1000.0]>,
 <Parameter 'b': 40000.0000 ± 4000.0000 cm^2, bounds=[10000.0:1000000.0]>)

In [7]:
a.min=-10
a.value=-2
a,b

(<Parameter 'a': -2.0000 ± 10.0000 cm, bounds=[-10.0:1000.0]>,
 <Parameter 'b': -2.0000 ± 10.0000 cm, bounds=[-10.0:1000.0]>)

In `EasyScience`, all objects have a unique identifier: Their unique_names. These are set automatically but can also be set or changed manually:

In [8]:
a.unique_name, b.unique_name

('Parameter_0', 'Parameter_4')

In [9]:
a.unique_name = "easy_name"
a.unique_name

'easy_name'

Unique_names can be used directly in a dependency expression when encapsulated by quotes. When unique_names are used, the dependency_map is not needed.

In [10]:
b.make_dependent_on(dependency_expression="'easy_name'**2")
a,b

(<Parameter 'a': -2.0000 ± 10.0000 cm, bounds=[-10.0:1000.0]>,
 <Parameter 'b': 4.0000 ± 40.0000 cm^2, bounds=[0.0:1000000.0]>)

Parameters can also freely be changed between being dependent and independent again:

In [11]:
b.make_independent()
a.make_dependent_on(dependency_expression="b/2", dependency_map={"b": b})
a,b

(<Parameter 'a': 2.0000 ± 20.0000 cm^2, bounds=[0.0:500000.0]>,
 <Parameter 'b': 4.0000 ± 40.0000 cm^2, bounds=[0.0:1000000.0]>)

In [12]:
b.convert_unit("m^2")
a,b

(<Parameter 'a': 0.0002 ± 0.0020 m^2, bounds=[0.0:50.0]>,
 <Parameter 'b': 0.0004 ± 0.0040 m^2, bounds=[0.0:100.0]>)

If you by mistake make a cyclic dependency between parameters, this is caught and triggers an error:

In [13]:
b.make_dependent_on(dependency_expression="a**2", dependency_map={"a": a})

RuntimeError: 
 Potential cyclic dependency detected!
This parameter, easy_name, has already been updated by Parameter_4 during this update.
Please check your dependencies.

# What can this be used for?

Dependent parameters have many use-cases, such as symmetry constraints for atomic lattices:

In [47]:
side_a = Parameter(name="side_a", value=3.44, variance=0.01, unit="Å", min=0, max=np.inf)
side_b = Parameter.from_dependency(name="side_b", dependency_expression="side_a", dependency_map={"side_a": side_a})
side_c = Parameter.from_dependency(name="side_c", dependency_expression="2*side_a", dependency_map={"side_a": side_a})

Or implementing complex structures, such as atomic fractions:

In [14]:
fraction_a = Parameter(name="fraction_a", value=0.5, variance=0.01, unit="m", min=0, max=1)
full_fraction = DescriptorNumber(name="full_fraction", value=1.0, variance=0.01, unit="m")
fraction_b = Parameter.from_dependency(name="fraction_b", dependency_expression="full_fraction-fraction_a", dependency_map={"fraction_a": fraction_a, "full_fraction": full_fraction})

# What's next?

**_Arithmetic operations with Scipp arrays:_**

To make it much easier to define models using Scipp arrays as data-holders

**_Mathematical functions for Parameters:_**

Like Cosine and Sine, to extend the usability of depedent parameters as ease model creation

**_Serialization and deserialization for dependent Paramters_**

To allow for saving/loading dependent Parameters